In [18]:
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from bs4 import BeautifulSoup
import urllib
%matplotlib inline
import datetime as datetime

In [19]:
#Open page and parse UTML
page = urllib.request.urlopen('http://www.massdot.state.ma.us/highway/TrafficTravelResources/TrafficInformationMap/RealTimeTraffic.aspx').read()
soup = BeautifulSoup(page, "lxml")
#Find desired table
table = soup.find('table', attrs ={'class':'xmlView'})
#grab all table bodies (3 in total)
tbodies = table.find_all('tbody')

In [20]:
#get routes separately, since we only need to get the routes once
#Initilize list of routes
routes = []

for tbody in tbodies:
    #for each tbody, find all rows in the body
    rows = tbody.find_all('tr')
    for row in rows:
        entries = row.find_all('td')                
        if len(entries) == 3:
        #Excludes rows that just consist of "Rt. #"
            if entries[0].string is not None:
                #The first tag gives description of road location
                road = entries[0].string
                routes.append(road)

In [21]:
#Initilize list of traffic speed and travel time
speed = []
travel = []

def grab_data(speed_ls, travel_ls):
    for tbody in tbodies:
        #for each tbody, find all rows in the body
        rows = tbody.find_all('tr')

        for row in rows:
            entries = row.find_all('td')
            if len(entries) == 3:
            #Excludes rows that just consist of "Rt. #"
                if entries[1].string is not None:
                     #The second tag gives description of travel time
                    time = entries[1].string
                    travel_ls.append(time)  
                else:
                    #adds NA for blank travel times
                    travel_ls.append("NA")
                if entries[2].string is not None:
                    #The third tag gives description of traffic speed
                    s = entries[2].string
                    speed_ls.append(s)
grab_data(speed, travel)

In [22]:
def scrape():
    #Open page and parse UTML
    page = urllib.request.urlopen('http://www.massdot.state.ma.us/highway/TrafficTravelResources/TrafficInformationMap/RealTimeTraffic.aspx').read()
    soup = BeautifulSoup(page, "lxml")
    #Find desired table
    table = soup.find('table', attrs ={'class':'xmlView'})
    #grab all table bodies (3 in total)
    tbodies = table.find_all('tbody')
    
    speed = []
    travel = []
    grab_data(speed, travel)
    
    d = {"Travel Time at "+datetime.datetime.now().strftime("%d-%m-%y %H:%M"): travel,
     "Speed at " + datetime.datetime.now().strftime("%d-%m-%y %H:%M"): speed}
    scraped_df = pd.DataFrame(data=d)
    return scraped_df

In [23]:
d = {"Routes":routes,
     "Travel Time at "+ datetime.datetime.now().strftime("%d-%m-%y %H:%M"): travel,
     "Speed at " + datetime.datetime.now().strftime("%d-%m-%y %H:%M"): speed}
df = pd.DataFrame(data=d)

In [28]:
days_passed = 0
end_day = 2
time_not_expired = True

initial = int(datetime.datetime.now().strftime("%M"))
initial_day = int(datetime.datetime.now().strftime("%M"))
prev_min = initial

#picks out minute part of the datetime element
while time_not_expired:
    current = int(datetime.datetime.now().strftime("%M"))
    current_day = int(datetime.datetime.now().strftime("%M"))
    #picks out minute part of the datetime element
    if prev_min != current:
        #scrape when a minute has passed
        new_df = scrape()
        print (datetime.datetime.now().strftime("%d-%m-%y %H:%M"))
        df = df.join(new_df)
    prev_min = current
    if current_day == (initial_day + 2):
        df.to_csv("traffic for "+ str(initial) + "minute.csv", encoding='utf-8')
        days_passed += 1
        initial_day = current_day
    time_not_expired = (days_passed != end_day)
print('done')

16-06-17 13:39
16-06-17 13:40
16-06-17 13:41
16-06-17 13:42
done


In [ ]:
#code for hourly scripting for a week

initial_hr = int(datetime.datetime.now().strftime("%H"))
#picks out hour part of the datetime element
initial_day = int(datetime.datetime.now().strftime("%d"))
#saves start date

end_day = 7
#run for 7 days
days_passed = 0
#tracks days passed
time_not_expired = True

hr = initial_hr
while time_not_expired:
    current_hr = int(datetime.datetime.now().strftime("%H"))
    current_day = int(datetime.datetime.now().strftime("%d"))
    #picks out minute part of the datetime element
    if hr != current_hr:
        #scrape when an hour has passed
        new_df = scrape()
        print (datetime.datetime.now().strftime("%d-%m-%y %H:%M"))
        df = df.join(new_df)
    hr = current_hr
    if current_day == (initial_day + 1):
        df.to_csv("traffic for "+ str(initial_day) +".csv", encoding='utf-8')
        #write df as a csv for each day
        days_passed += 1
        initial_day = current_day
        #update days passed
    time_not_expired =  (days_passed != end_day)

    
print('done')

In [28]:
#testing for every minute (= hours) and for 1 hour (= 1 days)

end_hour = 1
#run for 2 minutes
hour_passed = 0
#tracks min passed
time_not_expired = True

initial_min = int(datetime.datetime.now().strftime("%M"))
initial_hour = int(datetime.datetime.now().strftime("%H"))
#saves start min

minute = initial_minute

while time_not_expired:
    current_min = int(datetime.datetime.now().strftime("%M"))
    current_hour = int(datetime.datetime.now().strftime("%H"))
    #picks out minute part of the datetime element
    if minute == current_min+10:
        #scrape when an hour has passed
        new_df = scrape()
        print (datetime.datetime.now().strftime("%d-%m-%y %H:%M"))
        df = df.join(new_df)
    minute = current_minute
    if current_hour != initial_hour:
        hour_passed += 1
        df.to_csv("traffic after"+ str(hour_passed) +"hour.csv", encoding='utf-8')
        #write df as a csv for each day
        #update days passed
    time_not_expired =  (hour_passed != end_hour)

    
print('done')

done


In [37]:
pd.read_csv("traffic on 13.csv")

,Unnamed: 0,Routes,Speed at 16-06-17 18:11,Travel Time at 16-06-17 18:11,Speed at 16-06-17 18:12,Travel Time at 16-06-17 18:12,Speed at 16-06-17 18:13,Travel Time at 16-06-17 18:13,Speed at 16-06-17 18:14,Travel Time at 16-06-17 18:14,Speed at 16-06-17 18:15,Travel Time at 16-06-17 18:15
0,0,Yankee Division Hwy first roundabout after bri...,Greater than 50 mph,2 min 53 sec,Greater than 50 mph,2 min 53 sec,Greater than 50 mph,2 min 53 sec,Greater than 50 mph,2 min 53 sec,Greater than 50 mph,2 min 53 sec
1,1,Yankee Division Hwy before Exit 24 in Peabody ...,Greater than 50 mph,2 min 41 sec,Greater than 50 mph,2 min 41 sec,Greater than 50 mph,2 min 41 sec,Greater than 50 mph,2 min 41 sec,Greater than 50 mph,2 min 41 sec
2,2,Yankee Division Hwy before Exit 24 in Peabody ...,Greater than 50 mph,3 min 7 sec,Greater than 50 mph,3 min 7 sec,Greater than 50 mph,3 min 7 sec,Greater than 50 mph,3 min 7 sec,Greater than 50 mph,3 min 7 sec
3,3,Yankee Division Hwy before Exit 21 in Beverly ...,Greater than 50 mph,2 min 52 sec,Greater than 50 mph,2 min 52 sec,Greater than 50 mph,2 min 52 sec,Greater than 50 mph,2 min 52 sec,Greater than 50 mph,2 min 52 sec
4,4,Yankee Division Hwy before Exit 21 in Beverly ...,Greater than 50 mph,0 min 30 sec,Greater than 50 mph,0 min 30 sec,Greater than 50 mph,0 min 30 sec,Greater than 50 mph,0 min 30 sec,Greater than 50 mph,0 min 30 sec
5,5,Yankee Division Hwy before Exit 15 in Essex to...,Greater than 50 mph,7 min 31 sec,Greater than 50 mph,7 min 31 sec,Greater than 50 mph,7 min 31 sec,Greater than 50 mph,7 min 31 sec,Greater than 50 mph,7 min 31 sec
6,6,Yankee Division Hwy before Exit 15 in Essex to...,Greater than 50 mph,1 min 49 sec,Greater than 50 mph,1 min 49 sec,Greater than 50 mph,1 min 49 sec,Greater than 50 mph,1 min 49 sec,Greater than 50 mph,1 min 49 sec
7,7,Yankee Division Hwy after Exit 20B in Beverly ...,Greater than 50 mph,0 min 35 sec,Greater than 50 mph,0 min 35 sec,Greater than 50 mph,0 min 35 sec,Greater than 50 mph,0 min 35 sec,Greater than 50 mph,0 min 35 sec
8,8,Yankee Division Hwy after Exit 20B in Beverly ...,Greater than 50 mph,8 min 4 sec,Greater than 50 mph,8 min 4 sec,Greater than 50 mph,8 min 4 sec,Greater than 50 mph,8 min 4 sec,Greater than 50 mph,8 min 4 sec
9,9,Yankee Division Hwy after Exit 14 in Glouceste...,Greater than 50 mph,2 min 35 sec,Greater than 50 mph,2 min 35 sec,Greater than 50 mph,2 min 35 sec,Greater than 50 mph,2 min 35 sec,Greater than 50 mph,2 min 35 sec
